In [1]:
!pip install spotipy
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 63 kB 1.3 MB/s 
     |████████████████████████████████| 138 kB 13.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests~=2.23.0, but you have requests 2.27.1 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials

In [3]:
cid = 'XXXXsecret = 'xxxx'
redirect_uri='https://localhost:7777/callback'
username = 'xxxx'

In [5]:
client_credentials_manager = SpotifyClientCredentials(client_id='CLIENT_ID', client_secret='CLIENT_SECRET')
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [6]:
artist_name = []
track_name = []
track_popularity = []
artist_id = []
track_id = []
for i in range(0,1000,50):
    track_results = sp.search(q='year:2021', type='track', limit=50,offset=i)
    for i, t in enumerate(track_results['tracks']['items']):
        artist_name.append(t['artists'][0]['name'])
        artist_id.append(t['artists'][0]['id'])
        track_name.append(t['name'])
        track_id.append(t['id'])
        track_popularity.append(t['popularity'])

In [7]:
track_df = pd.DataFrame({'artist_name' : artist_name, 'track_name' : track_name, 'track_id' : track_id, 'track_popularity' : track_popularity, 'artist_id' : artist_id})
print(track_df.shape)
track_df.head()

(1000, 5)


,artist_name,track_name,track_id,track_popularity,artist_id
0,Doja Cat,Woman,6Uj1ctrBOjOas8xZXGqKk4,91,5cj0lLjcoR7YOSnhnX0Po5
1,Mr.Kitty,Years - 2021 Remaster,4S4ZY1yKo3WUtzsg3O6hcf,22,0pWwt5vGNzezEhfAcc420Y
2,Olivia Rodrigo,good 4 u,4ZtFanR9U6ndgddUvNcjcG,90,1McMsnEElThX1knmY4oliG
3,Elvis Costello & The Attractions,Pump It Up - 2021 Remaster,3oyc1mIdCBGaU55wX7otqM,61,4qmHkMxr6pTWh5Zo74odpH
4,Morgan Wallen,Wasted On You,3cBsEDNhFI9E82vPj3kvi3,82,4oUHIQIBe0LHzYfvXNW4QM


In [8]:
artist_popularity = []
artist_genres = []
artist_followers = []
for a_id in track_df.artist_id:
  artist = sp.artist(a_id)
  artist_popularity.append(artist['popularity'])
  artist_genres.append(artist['genres'])
  artist_followers.append(artist['followers']['total'])

In [9]:

track_df = track_df.assign(artist_popularity=artist_popularity, artist_genres=artist_genres, artist_followers=artist_followers)
track_df.head()

,artist_name,track_name,track_id,track_popularity,artist_id,artist_popularity,artist_genres,artist_followers
0,Doja Cat,Woman,6Uj1ctrBOjOas8xZXGqKk4,91,5cj0lLjcoR7YOSnhnX0Po5,88,"[dance pop, pop]",19304831
1,Mr.Kitty,Years - 2021 Remaster,4S4ZY1yKo3WUtzsg3O6hcf,22,0pWwt5vGNzezEhfAcc420Y,69,[weirdcore],311239
2,Olivia Rodrigo,good 4 u,4ZtFanR9U6ndgddUvNcjcG,90,1McMsnEElThX1knmY4oliG,85,[pop],17414255
3,Elvis Costello & The Attractions,Pump It Up - 2021 Remaster,3oyc1mIdCBGaU55wX7otqM,61,4qmHkMxr6pTWh5Zo74odpH,53,"[art rock, folk rock, mellow gold, new wave po...",230388
4,Morgan Wallen,Wasted On You,3cBsEDNhFI9E82vPj3kvi3,82,4oUHIQIBe0LHzYfvXNW4QM,83,[contemporary country],3455309


In [10]:
track_features = []
for t_id in track_df['track_id']:
  af = sp.audio_features(t_id)
  track_features.append(af)
tf_df = pd.DataFrame(columns = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature'])
for item in track_features:
  for feat in item:
    tf_df = tf_df.append(feat, ignore_index=True)
tf_df.head()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,0.824,0.764,5,-4.175,0,0.0854,0.088800,0.002940,0.1170,0.881,107.998,audio_features,6Uj1ctrBOjOas8xZXGqKk4,spotify:track:6Uj1ctrBOjOas8xZXGqKk4,https://api.spotify.com/v1/tracks/6Uj1ctrBOjOa...,https://api.spotify.com/v1/audio-analysis/6Uj1...,172627,4
1,0.608,0.841,9,-8.354,1,0.0293,0.000672,0.000017,0.0704,0.185,129.994,audio_features,4S4ZY1yKo3WUtzsg3O6hcf,spotify:track:4S4ZY1yKo3WUtzsg3O6hcf,https://api.spotify.com/v1/tracks/4S4ZY1yKo3WU...,https://api.spotify.com/v1/audio-analysis/4S4Z...,214159,4
2,0.563,0.664,9,-5.044,1,0.1540,0.335000,0.000000,0.0849,0.688,166.928,audio_features,4ZtFanR9U6ndgddUvNcjcG,spotify:track:4ZtFanR9U6ndgddUvNcjcG,https://api.spotify.com/v1/tracks/4ZtFanR9U6nd...,https://api.spotify.com/v1/audio-analysis/4ZtF...,178147,4
3,0.645,0.809,11,-6.120,1,0.0385,0.009210,0.001080,0.1060,0.966,138.978,audio_features,3oyc1mIdCBGaU55wX7otqM,spotify:track:3oyc1mIdCBGaU55wX7otqM,https://api.spotify.com/v1/tracks/3oyc1mIdCBGa...,https://api.spotify.com/v1/audio-analysis/3oyc...,196680,4
4,0.505,0.657,11,-5.240,0,0.0318,0.373000,0.001070,0.1260,0.252,196.000,audio_features,3cBsEDNhFI9E82vPj3kvi3,spotify:track:3cBsEDNhFI9E82vPj3kvi3,https://api.spotify.com/v1/tracks/3cBsEDNhFI9E...,https://api.spotify.com/v1/audio-analysis/3cBs...,178520,3


In [11]:
cols_to_drop2 = ['key','mode','type', 'uri','track_href','analysis_url']
tf_df = tf_df.drop(columns=cols_to_drop2)
print(track_df.info())
print(tf_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist_name        1000 non-null   object
 1   track_name         1000 non-null   object
 2   track_id           1000 non-null   object
 3   track_popularity   1000 non-null   int64 
 4   artist_id          1000 non-null   object
 5   artist_popularity  1000 non-null   int64 
 6   artist_genres      1000 non-null   object
 7   artist_followers   1000 non-null   int64 
dtypes: int64(3), object(5)
memory usage: 62.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   danceability      1000 non-null   float64
 1   energy            1000 non-null   float64
 2   loudness          1000 non-null   float64
 3   speechiness       1000 

In [12]:
track_df['artist_name'] = track_df['artist_name'].astype("string")
track_df['track_name'] = track_df['track_name'].astype("string")
track_df['track_id'] = track_df['track_id'].astype("string")
track_df['artist_id'] = track_df['artist_id'].astype("string")
tf_df['duration_ms'] = pd.to_numeric(tf_df['duration_ms'])
tf_df['instrumentalness'] = pd.to_numeric(tf_df['instrumentalness'])
tf_df['time_signature'] = tf_df['time_signature'].astype("category")
print(track_df.info())
print(tf_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 8 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   artist_name        1000 non-null   string
 1   track_name         1000 non-null   string
 2   track_id           1000 non-null   string
 3   track_popularity   1000 non-null   int64 
 4   artist_id          1000 non-null   string
 5   artist_popularity  1000 non-null   int64 
 6   artist_genres      1000 non-null   object
 7   artist_followers   1000 non-null   int64 
dtypes: int64(3), object(1), string(4)
memory usage: 62.6+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype   
---  ------            --------------  -----   
 0   danceability      1000 non-null   float64 
 1   energy            1000 non-null   float64 
 2   loudness          1000 non-null   float64 
 3   speechi

In [17]:
track_df.sort_values(by=['track_popularity'], ascending=False)[['track_name', 'artist_name']].head(20)

,track_name,artist_name
0,Woman,Doja Cat
104,Where Are You Now,Lost Frequencies
36,abcdefu,GAYLE
14,Enemy (with JID) - from the series Arcane Leag...,Imagine Dragons
2,good 4 u,Olivia Rodrigo
164,Envolver,Anitta
34,Shivers,Ed Sheeran
346,Jordan,Ryan Castro
8,THATS WHAT I WANT,Lil Nas X
54,Bad Habits,Ed Sheeran


In [18]:
by_art_fol = pd.DataFrame(track_df.sort_values(by=['artist_followers'], ascending=False)[['artist_followers','artist_popularity', 'artist_name','artist_genres']])
by_art_fol.astype(str).drop_duplicates().head(20)

,artist_followers,artist_popularity,artist_name,artist_genres
54,98070592,91,Ed Sheeran,"['pop', 'uk pop']"
484,63889939,93,Drake,"['canadian hip hop', 'canadian pop', 'hip hop'..."
678,63410911,86,Billie Eilish,"['art pop', 'electropop', 'pop']"
114,61484524,91,Justin Bieber,"['canadian pop', 'pop']"
908,56203982,89,Eminem,"['detroit hip hop', 'hip hop', 'rap']"
534,53904525,92,Taylor Swift,['pop']
608,50345863,90,BTS,"['k-pop', 'k-pop boy group']"
76,49422902,100,Bad Bunny,"['latin', 'reggaeton', 'trap latino']"
934,45468129,91,The Weeknd,"['canadian contemporary r&b', 'canadian pop', ..."
160,40912202,86,Bruno Mars,"['dance pop', 'pop']"


In [19]:
def to_1D(series):
 return pd.Series([x for _list in series for x in _list])
to_1D(track_df['artist_genres']).value_counts().head(20)

progressive house       171
trance                  157
pop                     151
uplifting trance        132
progressive trance      123
edm                     112
pop dance               102
rap                      95
contemporary country     61
dance pop                59
hip hop                  56
rock                     54
art rock                 49
mellow gold              49
trap                     48
folk rock                47
new wave pop             45
pub rock                 45
power pop                45
canadian pop             29
dtype: int64

In [20]:
top_10_genres = list(to_1D(track_df['artist_genres']).value_counts().index[:20])
top_artists_by_genre = []
for genre in top_10_genres:
  for index, row in by_art_fol.iterrows():
    if genre in row['artist_genres']:
      top_artists_by_genre.append({'artist_name':row['artist_name'], 'artist_genre':genre})
      break
pd.json_normalize(top_artists_by_genre)

,artist_name,artist_genre
0,Armin van Buuren,progressive house
1,Tiësto,trance
2,Ed Sheeran,pop
3,Above & Beyond,uplifting trance
4,Above & Beyond,progressive trance
5,Marshmello,edm
6,Marshmello,pop dance
7,Drake,rap
8,Luke Combs,contemporary country
9,Bruno Mars,dance pop
